In [1]:
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [2]:
reviews = []
for r in readGz("train.json.gz"):
    reviews.append(r)
print reviews[10]

{'rating': 3.0, 'reviewHash': 'R226572783', 'businessID': 'B634950425', 'unixReviewTime': 1321549271, 'reviewText': u"Great if you want a special night in a super plush environment but don't expect to pay less than $10 per drink.", 'userID': 'U369842280', 'reviewTime': u'Nov 17, 2011', 'categories': [u'Luxury Hotel', u'Wedding Venue']}


In [3]:
userList = []
businessList = []
#user->business hashtable
userHash = {}
#business->user hashtable
businessHash = {}
#list of pairs
pairs = []

for v in reviews:
    userID = v['userID']
    businessID = v['businessID']
    pairs.append((userID,businessID))
    if businessID not in businessHash:
        businessHash[businessID] = {}
        businessList.append(businessID)
    if userID not in userHash:
        userHash[userID] = {}
        userList.append(userID)
    if userID not in businessHash[businessID]:
        businessHash[businessID][userID] = 0
    if businessID not in userHash[userID]:
        userHash[userID][businessID] = 0
        
        
    userHash[userID][businessID] += 1
    businessHash[businessID][userID] += 1

import random
f_pairs = []
while len(f_pairs) != 200000:
    user = random.choice(userList)
    business = random.choice(businessList)
    if user not in businessHash[business]:
        f_pairs.append((user,business))
print userHash["U093387342"]
train_true_pairs = pairs[:100000]
train_false_pairs = f_pairs[:100000]
valid_true_pairs = pairs[100000:]
valid_false_pairs = f_pairs[100000:]

{'B211117692': 1, 'B744259798': 1, 'B478307368': 1, 'B915123517': 1, 'B377220586': 1, 'B675368412': 1, 'B161779289': 1, 'B652388772': 1, 'B408037852': 1, 'B038663558': 1, 'B123499039': 1, 'B274985615': 1, 'B433549047': 1, 'B071971841': 1, 'B940390363': 1, 'B266571746': 1, 'B716109571': 1, 'B311598637': 1, 'B433899722': 1, 'B775853642': 1}


In [4]:
print len(userList)
print len(businessList)

18793
21321


In [35]:
import numpy as np
n_user = 6000#len(userList)
n_business = 6000#len(businessList)
UBArr = np.empty([len(userList),n_business])
BUArr = np.empty([len(businessList),n_user])

for i in range(len(userList)):
    if i%1000 == 0:
        print i
    user = userList[i]
    for j in range(n_business):
        business = businessList[j]        
        if business in userHash[user]:
            UBArr[i][j] = 1
        else:
            UBArr[i][j] = 0
            
for i in range(len(businessList)):
    if i%1000 == 0:
        print i
    business = businessList[i]
    for j in range(n_user):
        user = userList[j]        
        if user in businessHash[business]:
            BUArr[i][j] = 1
        else:
            BUArr[i][j] = 0

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000


In [38]:
from sklearn.decomposition import PCA

userFac = PCA(n_components = 200)
businessFac = PCA(n_components = 200)

In [39]:
userFac.fit(UBArr)

PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [40]:
businessFac.fit(BUArr)

PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [41]:
lfu = userFac.transform(UBArr)
lfb = businessFac.transform(BUArr)

In [42]:
for k, v in zip(userList,lfu):
    userHash[k]['lf'] = v
for k, v in zip(businessList,lfb):
    businessHash[k]['lf'] = v
    
print reviews[0]
print businessHash['B408037852']['lf']

default_user = userFac.transform([[0] * n_business])[0]

default_business = userFac.transform([[0] * n_user])[0]

{'rating': 5.0, 'reviewHash': 'R471510664', 'businessID': 'B408037852', 'unixReviewTime': 1364143460, 'reviewText': u"This is where i go to shop for gifts from my mom. She loves this stuff. Cna't get enough. I like that you can customize the items. Store is well alid out and shoppable.", 'userID': 'U093387342', 'reviewTime': u'Mar 24, 2013', 'categories': [u"Women's Clothing Store", u'Fashion Accessories Store', u'Shoe Store']}
[ -6.61698866e-02   8.15300015e-02  -2.27832412e-02  -1.70265651e-02
  -1.04132455e-01   8.89245828e-02   3.91501323e-02  -1.16904748e-01
   4.02887300e-02  -1.55623418e-02   1.09357400e-02   1.62554772e-01
  -3.83522329e-02  -5.98752061e-03  -1.69697865e-02  -8.30643114e-02
  -1.86481173e-01  -1.03544336e-01   1.35905128e-02  -2.94708197e-02
  -2.05777521e-02  -8.11593323e-04   1.48013035e-02  -1.72954648e-01
  -1.10165095e-01   4.02819698e-01   2.41599298e-01   1.60541500e-01
   1.67263960e-01   1.26879179e-01   1.75777626e-01   3.64068414e-01
   2.36123617e-0

In [47]:
def getFeats(pair):
    user, business = pair
    if user in userHash:
        userF = userHash[user]['lf']
    else:
        userF = default_user
    if business in businessHash:
        businessF = businessHash[business]['lf']
    else:
        businessF = default_business
    return userF + businessF

In [48]:
X = [getFeats(p) for p in train_true_pairs +valid_true_pairs + valid_false_pairs+ train_false_pairs]
y = [1] * 200000 + [0] * 200000

In [49]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(early_stopping = True, verbose = True)
classifier.fit(X,y)

Iteration 1, loss = 0.55243393
Validation score: 0.762700
Iteration 2, loss = 0.45242387
Validation score: 0.801975
Iteration 3, loss = 0.40396831
Validation score: 0.816500
Iteration 4, loss = 0.37517602
Validation score: 0.829825
Iteration 5, loss = 0.35428593
Validation score: 0.837775
Iteration 6, loss = 0.33808496
Validation score: 0.840925
Iteration 7, loss = 0.32528015
Validation score: 0.844675
Iteration 8, loss = 0.31539901
Validation score: 0.847500
Iteration 9, loss = 0.30550236
Validation score: 0.848675
Iteration 10, loss = 0.29824050
Validation score: 0.851450
Iteration 11, loss = 0.29141485
Validation score: 0.849950
Iteration 12, loss = 0.28532392
Validation score: 0.852300
Iteration 13, loss = 0.28023146
Validation score: 0.853050
Iteration 14, loss = 0.27511489
Validation score: 0.853175
Iteration 15, loss = 0.27119505
Validation score: 0.853600
Iteration 16, loss = 0.26696771
Validation score: 0.853500
Iteration 17, loss = 0.26332694
Validation score: 0.853550
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [ ]:
def predict(pair):
    feat = [getFeats(pair)]
    return (classifier.predict(feat) == 1)

In [50]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if predict((u,i)):
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

In [51]:
print "wuwuwu"

wuwuwu
